In [0]:
import os
import time
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import plotly.express as px

In [0]:
# Load .env
load_dotenv()

# Load API-keys
ALPHA_API_KEY = os.getenv('ALPHAVANTAGE_API_KEY')
GBQ_API_KEY = os.getenv('GBQ_API_KEY')

In [0]:
# Set the symbols I want to request
symbols = [
     'VOO'
    ,'VGT'
    ,'AAPL'
    ,'MSFT'
    ,'GOOGL'
    ,'AMZN'
    ]

In [0]:
# Initialize DataFrames
my_stocks = pd.DataFrame(columns=['Date','Symbol','Price','Price_Perc_Day_Change','Perc_Change_From_Initial_Price'])
my_stocks_max_price = pd.DataFrame(columns=['Date','Symbol','Price','Price_Perc_Day_Change','Perc_Change_From_Initial_Price'])
my_stocks_initial_value = pd.DataFrame(columns=['Symbol','Initial Date', 'Initial Price'])


# Iterate through symbols
for symbol in symbols:
    
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={ALPHA_API_KEY}'
    
    # The AlphaVantage free API only allows 5 API calls every 60 seconds, so when a failed call occurs wait 10 seconds
    API_call_flag = True
    while API_call_flag:
        response = requests.get(url)
        if 'Note' in response.json():
            print(f'Sleeping 10 seconds on {symbol} to not overload AlphaVantage free API')
            time.sleep(10)
        else:
            API_call_flag = False
            print(f'Successfully fetched: {symbol}')

    # Get the successful API response's JSON key 'Time Series Daily'
    symbol_data = response.json()
    trading_data = symbol_data['Time Series (Daily)']

    # Load the API response per symbol's JSON key '4. close' into a DataFrame, transpose it, and rename it to the current symbol
    close_data_series = pd.to_numeric(pd.DataFrame(trading_data).T['4. close'].rename(symbol))

    # Get minimum date per stock and append to my_stocks_initial_value
    initial_date = close_data_series.index.min()
    initial_price = close_data_series[initial_date]
    my_stocks_initial_value = my_stocks_initial_value.append({'Symbol':symbol, 'Initial Date':initial_date, 'Initial Price':initial_price}, ignore_index=True)

    # Create and add series of evolution of price percentage increase as column to my_stocks
    this_symbol_perc_chg = round(((close_data_series / initial_price) - 1) * 100, 2)

    # Bring together data
    this_stock_df = pd.DataFrame(close_data_series).reset_index().rename(columns={'index':'Date',f'{symbol}':'Price'})
    this_stock_df['Symbol'] = symbol
    this_stock_df['Price_Perc_Day_Change'] = round(((this_stock_df['Price'] / this_stock_df['Price'].shift(-1)) - 1) * 100, 2)
    this_stock_df['Price_Perc_Day_Change'].fillna(0, inplace=True)
    this_stock_df['Perc_Change_From_Initial_Price'] = this_symbol_perc_chg.array

    # Append data to my_stocks
    my_stocks = my_stocks.append(this_stock_df)

# Reset my_stocks index to be a RangeIndex, instead of a repeated series
my_stocks.index = pd.RangeIndex(len(my_stocks.index))

# Declare success when all symbols have been successfully fetched
print('All symbols successfully fetched')

In [0]:
# Find the index of the max price per symbol, then append that row to the symbols_max_price DataFrame
symbols_max_price = my_stocks.groupby(['Symbol']).idxmax()['Price']

for idx in symbols_max_price:
    my_stocks_max_price = my_stocks_max_price.append(my_stocks.iloc[idx,:])

my_stocks_max_price.rename(columns={'Price':'MAX_Price'}, inplace=True)

In [0]:
my_stocks

In [0]:
my_stocks_initial_value

In [0]:
my_stocks_max_price

In [0]:
# Save results to .csv's
my_stocks.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks')
my_stocks_initial_value.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks_initial_Value')
my_stocks_max_price.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks_max_price')